Calcul le coefficient de silhouette pour une distance cosine/euclidean pour les X% d'activés les plus proches entre eux, en regardant les 10% les plus proches (activés/non activés pour calculer le coef)

Temps environ: 20h

In [43]:
#IPython extension to reload modules before executing user code.
#'autoreload' reloads modules automatically before entering the execution of code typed at the IPython prompt.
%load_ext autoreload
%autoreload 2
import os
os.environ['PYTHONHASHSEED'] = '0'
import gpustat

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
#select the best free GPU on the nvidia card
stats = gpustat.GPUStatCollection.new_query()
ids = map(lambda gpu: int(gpu.entry['index']), stats)
ratios = map(lambda gpu: float(gpu.entry['memory.used'])/float(gpu.entry['memory.total']), stats)
bestGPU = min(zip(ids, ratios), key=lambda x: x[1])[0]
bestGPU = 3

print("setGPU: Setting GPU to: {}".format(bestGPU))
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = str(bestGPU)

'''
#set memory usage to 0.5
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
set_session(tf.Session(config=config))'''

setGPU: Setting GPU to: 3


'\n#set memory usage to 0.5\nfrom keras.backend.tensorflow_backend import set_session\nimport tensorflow as tf\nconfig = tf.ConfigProto()\nconfig.gpu_options.per_process_gpu_memory_fraction = 0.5\nset_session(tf.Session(config=config))'

In [45]:
#!pip install -q sklearn
import collections
import numpy as np, numpy
from keract import get_activations, display_activations
import keras
from keras import backend as K
from keras import optimizers, regularizers
from keras.callbacks import LearningRateScheduler
from keras.datasets import cifar10, cifar100 # we can use also cifar100
from keras.layers import Input, BatchNormalization, AveragePooling2D, ZeroPadding2D, LeakyReLU, GlobalAveragePooling2D, Dense, Flatten, Activation, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential, load_model, Model
from keras.regularizers import l2
from keras.utils import plot_model, to_categorical
from keras.utils.vis_utils import model_to_dot
import operator 
import matplotlib.pyplot as plt
import scipy
import sklearn
import sklearn.metrics
import sys
#sys.executable
#sys.path
import time
from IPython.display import SVG

In [46]:
def normalize(x, mean, std):
    # This function normalizes inputs for zero mean and unit variance to speed up learning.
    
    # In case std = 0, we add eps = 1e-7
    eps = K.epsilon()
    x = (x-mean)/(std+eps)
    return x
  
def import_cifar(dataset):
    if dataset == 10:
        (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    elif dataset == 100:
        (x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

    # By default, they are uint8 but we need them float to normalize them
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    
    # Calculating the mean and standard deviation of the training data
    mean = np.mean(x_train, axis=(0, 1, 2, 3))
    std = np.std(x_train, axis=(0, 1, 2, 3))
    
    # Normalizing 
    x_train = normalize(x_train, mean, std)
    x_test = normalize(x_test, mean, std)

    y_train = to_categorical(y_train, num_classes=dataset)
    y_test = to_categorical(y_test,  num_classes=dataset)
    
    return (x_train, y_train), (x_test, y_test)

In [47]:
# LOAD DATABase
num_classes = 10
(x_train, y_train), (x_test, y_test) = import_cifar(num_classes)

In [48]:
# Architecture taken from https://github.com/geifmany/cifar-vgg
# Weight decay and Dropout have been removed
# BatchNormalization before activations
def VGG16_Vanilla_beta(input_shape, num_classes):
    model = Sequential([
        #0
        Conv2D(64, (3, 3), padding='same', input_shape=input_shape),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #3
        Conv2D(64, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        #7
        Conv2D(128, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #10
        Conv2D(128, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        #14
        Conv2D(256, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #17
        Conv2D(256, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #20
        Conv2D(256, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        #24
        Conv2D(512, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #27
        Conv2D(512, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #30
        Conv2D(512, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        #34
        Conv2D(512, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #37
        Conv2D(512, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #40
        Conv2D(512, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        #45
        Dense(512),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #48
        Dense(num_classes),
        BatchNormalization(scale=False, center=False),
        Activation('softmax')])
    return model

# CLUSTERING

In [49]:
# we suppose k = 2 (activé and inactivé)

def AverageDistance(vecteur,vecteurs_without_vecteur):
    nCluster_i = len(vecteurs_without_vecteur)
    distance = 0
    for vect in vecteurs_without_vecteur:
        #distance = distance + scipy.spatial.distance.euclidean(vecteur,vect)
        distance = distance + numpy.linalg.norm(vecteur-vect)
    a = (1/(nCluster_i))*distance
    return a

def AverageDissimilarity(vecteur, vecteurs_from_another_cluster):
    nCluster_j = len(vecteurs_from_another_cluster)
    distance = 0
    for vect in vecteurs_from_another_cluster:
        #distance = distance + scipy.spatial.distance.euclidean(vecteur,vect)
        distance = distance + numpy.linalg.norm(vecteur-vect)
    b = (1/nCluster_j)*distance
    return b

def Silhouette(vecteurs, a, b):
    s = (b-a)/(max(a,b))
    return s

# quid si pas assez de partité ?
def SilhouetteScore(vecteurs_0,vecteurs_1):
    if len(vecteurs_0) == None or len(vecteurs_1) == None or len(vecteurs_0) == 0 or len(vecteurs_1) == 0:
        print('Error: all entries actived or not: Probably a pathologique neuron')
        return -2
    
    SilhouetteScoreSum = 0
    
    if len(vecteurs_0) < 3 :
        SilhouetteScoreSum = SilhouetteScoreSum + 1 #0
    else:
        for i in range(len(vecteurs_0)):
            a = AverageDistance(vecteurs_0[i], vecteurs_0[0:i]+vecteurs_0[(i+1):])
            b = AverageDissimilarity(vecteurs_0[i], vecteurs_1)
            s = Silhouette(vecteurs_0, a, b)
            SilhouetteScoreSum = SilhouetteScoreSum + s
    
    if len(vecteurs_1) < 2 :
        SilhouetteScoreSum = SilhouetteScoreSum + 1 #0
    else:
        for i in range(len(vecteurs_1)):
            a = AverageDistance(vecteurs_1[i], vecteurs_1[0:i]+vecteurs_1[(i+1):])
            b = AverageDissimilarity(vecteurs_1[i], vecteurs_0)
            s = Silhouette(vecteurs_1, a, b)
            SilhouetteScoreSum = SilhouetteScoreSum + s

    SilhouetteScore = SilhouetteScoreSum/(len(vecteurs_0)+len(vecteurs_1))
        
    return SilhouetteScore

In [50]:
def TakeTheNearest(Vector, Ratio, Dist):
    Length = len(Vector)
    DistVector = []
    if Dist == 'cosine':
        for i in range(len(Vector)):
            Dist = 0
            for j in range(len(Vector)):
                if i!=j:
                    Dist = Dist + scipy.spatial.distance.cosine(Vector[i],Vector[j])
            DistVector.append(Dist)
    elif Dist == 'euclidean':
        for i in range(len(Vector)):
            Dist = 0
            for j in range(len(Vector)):
                if i!=j:
                    Dist = Dist + scipy.spatial.distance.euclidean(Vector[i],Vector[j])
            DistVector.append(Dist)
    else:
        print("ERROR in the Distance specification")
    #print(len(Vector[0])) # nombre de dimension des activations
    #print(len(DistVector)) # nombre d'activations activé ou non-activé
    
    UltimeVector = zip(DistVector, Vector)
    UltimeVectorList = list(UltimeVector)
    #print(UltimeVectorList)
    #print(UltimeVectorList[0][0]) # 1ere distance totale
    #print(UltimeVectorList[0][1]) # coordonnées de la premiere activation
    #print(UltimeVectorList[:][1]) # toutes les coordonnées
    UltimeVectorList.sort(key=operator.itemgetter(0))
    #print(UltimeVectorList)
    #print(UltimeVectorList[0][0]) # distance 0
    #print(UltimeVectorList[0][1]) # coord 0
    #print(UltimeVectorList[1][0]) # dist 1
    
    Vec = [lis[1] for lis in UltimeVectorList]
    NewVector = Vec[:int(np.floor(Length*Ratio))]
    return NewVector

In [51]:
def CheckTheNearests(point, Vecteur, Label, Ratio, Dist):
    point = numpy.asarray([point])
    Vecteur = numpy.asarray(Vecteur)
    Length = len(Vecteur)
    Distances = scipy.spatial.distance.cdist(point, Vecteur, metric=Dist) #return a matrix
    
    UltimeVector = zip(Distances[0], Vecteur, Label)
    UltimeVectorList = list(UltimeVector)
    UltimeVectorList.sort(key=operator.itemgetter(0))
    Vec = [lis[1] for lis in UltimeVectorList]
    Vect = Vec[:int(np.floor(Length*Ratio))]
    Vec = [lis[2] for lis in UltimeVectorList]
    Lab = Vec[:int(np.floor(Length*Ratio))]

    return Vect, Lab

## Silhouette score mean by layer

In [52]:
def RUN(IM, modelename, Dist, Ratio1, Ratio2, NeuronsToTest):
    Silhouette_Score_Mean = []
    NbreOfNeurons = []
    NumberOfLayer = 13
    labelRatioMean = []
    for c in range(NumberOfLayer):
        print('\nCouche '+str(c+1))
        #Layer to test
        couche_name_2 = 'conv2d_{}/BiasAdd:0'.format(c+1)
        couche_name_3 = 'activation_{}/Relu:0'.format(c+1)

        NeuronsToTestIntoTheLayer = NeuronsToTest

        X_Kernel_size = len(IM[couche_name_2][0,:,:,:])
        Y_Kernel_size = len(IM[couche_name_2][0,0,:,:])
        Z_Kernel_size = len(IM[couche_name_2][0,0,0,:])

        Silhouette_Score_Sum = 0
        labelRatio = []

        for i in range(NeuronsToTestIntoTheLayer):
            print(' Neurone '+str(i))

            #Choix du neurone dans la couche
            X = int(numpy.ceil(numpy.random.rand(1)*X_Kernel_size)-1)
            Y = int(numpy.ceil(numpy.random.rand(1)*Y_Kernel_size)-1)
            Z = int(numpy.ceil(numpy.random.rand(1)*Z_Kernel_size)-1)
            #specification de ses bornes
            X_inf = X-1
            Y_inf = Y-1
            X_sup = X+1
            Y_sup = Y+1
            if X == 0: X_inf = 0
            if Y == 0: Y_inf = 0   
            if X == X_Kernel_size-1: X_sup = X_Kernel_size-1
            if Y == Y_Kernel_size-1: Y_sup = Y_Kernel_size-1

            Vecteurs_1 = [] #elements activés
            Vecteurs_All = [] #tous
            Labels_All = [] #leur label

            for j in range(nombreDImagesDActivation):
                if IM[couche_name_3][j,X,Y,Z] == 0.0:
                    Vecteurs_All.append(IM[couche_name_2][j,X_inf:X_sup,Y_inf:Y_sup,0:Z_Kernel_size].flatten())
                    Labels_All.append(0)
                elif IM[couche_name_3][j,X,Y,Z] > 0.0:
                    Vecteurs_1.append(IM[couche_name_2][j,X_inf:X_sup,Y_inf:Y_sup,0:Z_Kernel_size].flatten())
                    Vecteurs_All.append(IM[couche_name_2][j,X_inf:X_sup,Y_inf:Y_sup,0:Z_Kernel_size].flatten())
                    Labels_All.append(1)

            Vecteurs_1_nearest = TakeTheNearest(Vecteurs_1, Ratio1, Dist)
                
            #pour chaque element de Vecteurs_1_nearest
            Length_Vecteurs_1_nearest = len(Vecteurs_1_nearest)
            print('   Number of {} ration nearest activated element :'.format(Ratio1)+str(Length_Vecteurs_1_nearest))
            if Length_Vecteurs_1_nearest == 0:
                NeuronsToTestIntoTheLayer = NeuronsToTestIntoTheLayer -1
            else:
                labelUniqueError = 0
                S_Sum = 0
                for k in range(Length_Vecteurs_1_nearest):
                    Vect, Lab = CheckTheNearests(Vecteurs_1_nearest[k], Vecteurs_All, Labels_All, Ratio2, Dist)
                    #assert if there are strictly more than 1 label
                    if len(set(Lab)) >1:
                        S = sklearn.metrics.silhouette_score(Vect, Lab, metric=Dist)
                    else:
                        S = -2

                    if S != -2:
                        S_Sum = S_Sum + S
                    else:
                        labelUniqueError = labelUniqueError + 1
                    
                    # count the ratio of activated label around the activated point
                    labelRatio.append(Lab.count(1)/len(Lab))
                    #print('        Label activated ratio: '+str(labelRatio[-1]))
                
                labelErrorRatio = labelUniqueError/Length_Vecteurs_1_nearest
                print('   Label Unique Error into the '+str(Ratio2)+' ratio activated point area Ratio:'+str(labelErrorRatio))
        
                if labelUniqueError != Length_Vecteurs_1_nearest:
                    Silhouette_Score_Sum = Silhouette_Score_Sum + S_Sum/(Length_Vecteurs_1_nearest - labelUniqueError)
                else: 
                    NeuronsToTestIntoTheLayer = NeuronsToTestIntoTheLayer -1
                
        if NeuronsToTestIntoTheLayer != 0:
            Silhouette_Score_Mean.append(Silhouette_Score_Sum / NeuronsToTestIntoTheLayer)
            print(' => Silhouette_Score_Mean : '+str(Silhouette_Score_Mean[-1]))
        else:
            Silhouette_Score_Mean.append(0)
            print('  !!!  no neurons to test into this layer, only one label (activated or not)')
        NbreOfNeurons.append(NeuronsToTestIntoTheLayer)
        
        labelRatioMean.append(np.mean(labelRatio))
        
    print(Silhouette_Score_Mean)
    print(np.mean(Silhouette_Score_Mean))
    
    # plot:
    fig = plt.figure(figsize=(10, 5))
    plt.ylim(top=1)
    nombreCouches = len(Silhouette_Score_Mean)
    x = range(nombreCouches)

    markerline, stemlines, baseline = plt.stem(x, Silhouette_Score_Mean, markerfmt='o', label='model ')
    plt.setp(stemlines, 'color', plt.getp(markerline,'color'))
    plt.setp(stemlines, 'linestyle', 'dotted')
    plt.plot(np.mean(Silhouette_Score_Mean)*np.ones((nombreCouches,1)))
    for c in range(NumberOfLayer):
        plt.text(-0.27+c, 0.02, NbreOfNeurons[c], rotation= 90)

    #plt.legend()
    #plt.xlabel('Couches')
    #plt.xticks(np.arange(nombreCouches), ('Conv 1','Conv 2','Conv 3','Conv 4','Conv 5','Conv 6','Conv 7','Conv 8','Conv 9','Conv10','Conv11','Conv12','Conv13'), rotation=60)
    #plt.ylabel('Silhouette coef mean')
    #plt.title('Silhouette coef mean by layer for the model {}\n for {} activation images and {} neurons \n with Ratio1 {} and Ratio2 {}'.format(modelename,nombreDImagesDActivation,NeuronsToTest, Ratio1, Ratio2))
    plt.grid()
    #plt.rcParams.update({'font.size': 20})
    plt.show()
    fig.savefig('figures/SilhouetteCoef_Weighted/SilhouetteCoefMean_{}_Ratios{}_{}_{}neurons_{}.png'.format(modelename, Ratio1, Ratio2,NeuronsToTest, Dist), dpi=100) 
    
    
    # plot2 label ratio:
    fig = plt.figure(figsize=(10, 5))
    plt.ylim(top=1)
    nombreCouches = len(Silhouette_Score_Mean)
    x = range(nombreCouches)

    markerline, stemlines, baseline = plt.stem(x, labelRatioMean, markerfmt='o', label='model ')
    plt.setp(stemlines, 'color', plt.getp(markerline,'color'))
    plt.setp(stemlines, 'linestyle', 'dotted')
    plt.plot(np.mean(labelRatioMean)*np.ones((nombreCouches,1)))
    #plt.rcParams.update({'font.size': 20})
    plt.grid()
    #plt.xticks(np.arange(nombreCouches), ('Conv 1','Conv 2','Conv 3','Conv 4','Conv 5','Conv 6','Conv 7','Conv 8','Conv 9','Conv10','Conv11','Conv12','Conv13'), rotation=60)
    #plt.ylabel('label Ratio Mean')
    #plt.title('label Ratio Mean by layer for the model {}\n for {} activation images and {} neurons \n with Ratio1 {} and Ratio2 {}'.format(modelename,nombreDImagesDActivation,NeuronsToTest, Ratio1, Ratio2))
    plt.show()
    fig.savefig('figures/SilhouetteCoef_Weighted/labelRatioMean_{}_Ratios{}_{}_{}neurons_{}.png'.format(modelename, Ratio1, Ratio2,NeuronsToTest, Dist), dpi=100) 

In [53]:
# LOAD the model and get the activations
modelename = 'vgg16_wdecay_0'
model_name = '../weights/6/{}/initial/weights-initial.hdf5'.format(modelename) #final / initial ! attention au nom de sauvegarde !
modelename = 'initial'
model = VGG16_Vanilla_beta(input_shape=(32,32,3), num_classes=10)
model.load_weights('{}'.format(model_name))
sgd = optimizers.SGD(lr=0.002, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
Echantillon = []
nombreDImagesDActivation = 1000
choix = np.random.choice(x_test.shape[0], nombreDImagesDActivation)
Echantillon = x_train[choix, :, : , :]
IM = get_activations(model, Echantillon)# on affiche les noms des differentes couches

In [54]:
# print the activations infos
print(IM.keys())
model.summary()
## need to pick the ones that are weighted
#print(IM['conv2d_1/BiasAdd:0'][0,0,0,:]) #activation, X, Y, Z
#print(IM['batch_normalization_1/cond/Merge:0'][0,0,0,:]) #activation, X, Y, Z
#print(IM['activation_1/Relu:0'][0,0,0,:]) #activation, X, Y, Z
#print(IM['conv2d_2/BiasAdd:0'][0,0,0,:]) #activation, X, Y, Z

dict_keys(['activation_21/Relu:0', 'conv2d_15/BiasAdd:0', 'batch_normalization_25/cond/Merge:0', 'activation_22/Relu:0', 'conv2d_19/BiasAdd:0', 'activation_28/Relu:0', 'conv2d_14/BiasAdd:0', 'conv2d_20/BiasAdd:0', 'batch_normalization_29/cond/Merge:0', 'activation_18/Relu:0', 'batch_normalization_21/cond/Merge:0', 'batch_normalization_19/cond/Merge:0', 'dense_3/BiasAdd:0', 'activation_26/Relu:0', 'batch_normalization_26/cond/Merge:0', 'flatten_2/Reshape:0', 'conv2d_25/BiasAdd:0', 'activation_20/Relu:0', 'activation_30/Softmax:0', 'batch_normalization_30/cond/Merge:0', 'conv2d_17/BiasAdd:0', 'conv2d_26/BiasAdd:0', 'batch_normalization_18/cond/Merge:0', 'batch_normalization_27/cond/Merge:0', 'max_pooling2d_7/MaxPool:0', 'activation_23/Relu:0', 'activation_27/Relu:0', 'conv2d_18/BiasAdd:0', 'batch_normalization_24/cond/Merge:0', 'max_pooling2d_10/MaxPool:0', 'conv2d_21/BiasAdd:0', 'max_pooling2d_8/MaxPool:0', 'batch_normalization_17/cond/Merge:0', 'activation_19/Relu:0', 'batch_normalizat

In [ ]:
RATIO1 = 0.05
RATIO2 = 0.05
NEURONSTOTEST = 20

In [ ]:
# Run the code
RUN(IM, modelename, Dist='cosine', Ratio1=RATIO1, Ratio2=RATIO2, NeuronsToTest=NEURONSTOTEST)

In [ ]:
RUN(IM, modelename, Dist='euclidean', Ratio1=RATIO1, Ratio2=RATIO2, NeuronsToTest=NEURONSTOTEST) # diminuer de moitié le temps estimer si ok 10 neurones

In [ ]:
modelename = 'vgg16_layca_0'
model_name = '../weights/6/{}/final/weights-final.hdf5'.format(modelename)
model.load_weights('{}'.format(model_name))
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
IM = get_activations(model, Echantillon)

In [ ]:
RUN(IM, modelename, Dist='cosine', Ratio1=RATIO1, Ratio2=RATIO2, NeuronsToTest=NEURONSTOTEST)

In [ ]:
RUN(IM, modelename, Dist='euclidean', Ratio1=RATIO1, Ratio2=RATIO2, NeuronsToTest=NEURONSTOTEST)

In [ ]:
modelename = 'vgg16_layca_1'
model_name = '../weights/6/{}/final/weights-final.hdf5'.format(modelename)
model.load_weights('{}'.format(model_name))
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
IM = get_activations(model, Echantillon)

In [ ]:
RUN(IM, modelename, Dist='cosine', Ratio1=RATIO1, Ratio2=RATIO2, NeuronsToTest=NEURONSTOTEST)

In [ ]:
RUN(IM, modelename, Dist='euclidean', Ratio1=RATIO1, Ratio2=RATIO2, NeuronsToTest=NEURONSTOTEST)

In [ ]:
modelename = 'vgg16_layca_2'
model_name = '../weights/6/{}/final/weights-final.hdf5'.format(modelename)
model.load_weights('{}'.format(model_name))
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
IM = get_activations(model, Echantillon)

In [ ]:
RUN(IM, modelename, Dist='cosine', Ratio1=RATIO1, Ratio2=RATIO2, NeuronsToTest=NEURONSTOTEST)

In [ ]:
RUN(IM, modelename, Dist='euclidean', Ratio1=RATIO1, Ratio2=RATIO2, NeuronsToTest=NEURONSTOTEST)

In [ ]:
modelename = 'vgg16_sgd_0'
model_name = '../weights/6/{}/final/weights-final.hdf5'.format(modelename)
model.load_weights('{}'.format(model_name))
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
IM = get_activations(model, Echantillon)

In [ ]:
RUN(IM, modelename, Dist='cosine', Ratio1=RATIO1, Ratio2=RATIO2, NeuronsToTest=NEURONSTOTEST)

In [ ]:
RUN(IM, modelename, Dist='euclidean', Ratio1=RATIO1, Ratio2=RATIO2, NeuronsToTest=NEURONSTOTEST)

In [ ]:
modelename = 'vgg16_sgd_1'
model_name = '../weights/6/{}/final/weights-final.hdf5'.format(modelename)
model.load_weights('{}'.format(model_name))
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
IM = get_activations(model, Echantillon)

In [ ]:
RUN(IM, modelename, Dist='cosine', Ratio1=RATIO1, Ratio2=RATIO2, NeuronsToTest=NEURONSTOTEST)

In [ ]:
RUN(IM, modelename, Dist='euclidean', Ratio1=RATIO1, Ratio2=RATIO2, NeuronsToTest=NEURONSTOTEST)

In [ ]:
modelename = 'vgg16_sgd_2'
model_name = '../weights/6/{}/final/weights-final.hdf5'.format(modelename)
model.load_weights('{}'.format(model_name))
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
IM = get_activations(model, Echantillon)

In [ ]:
RUN(IM, modelename, Dist='cosine', Ratio1=RATIO1, Ratio2=RATIO2, NeuronsToTest=NEURONSTOTEST)

In [ ]:
RUN(IM, modelename, Dist='euclidean', Ratio1=RATIO1, Ratio2=RATIO2, NeuronsToTest=NEURONSTOTEST)

In [ ]:
modelename = 'vgg16_wdecay_0'
model_name = '../weights/6/{}/final/weights-final.hdf5'.format(modelename)
model.load_weights('{}'.format(model_name))
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
IM = get_activations(model, Echantillon)

In [ ]:
RUN(IM, modelename, Dist='cosine', Ratio1=RATIO1, Ratio2=RATIO2, NeuronsToTest=NEURONSTOTEST)

In [ ]:
RUN(IM, modelename, Dist='euclidean', Ratio1=RATIO1, Ratio2=RATIO2, NeuronsToTest=NEURONSTOTEST)

In [ ]:
modelename = 'vgg16_wdecay_1'
model_name = '../weights/6/{}/final/weights-final.hdf5'.format(modelename)
model.load_weights('{}'.format(model_name))
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
IM = get_activations(model, Echantillon)

In [ ]:
RUN(IM, modelename, Dist='cosine', Ratio1=RATIO1, Ratio2=RATIO2, NeuronsToTest=NEURONSTOTEST)

In [ ]:
RUN(IM, modelename, Dist='euclidean', Ratio1=RATIO1, Ratio2=RATIO2, NeuronsToTest=NEURONSTOTEST)

In [ ]:
modelename = 'vgg16_wdecay_2'
model_name = '../weights/6/{}/final/weights-final.hdf5'.format(modelename)
model.load_weights('{}'.format(model_name))
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
IM = get_activations(model, Echantillon)

In [ ]:
RUN(IM, modelename, Dist='cosine', Ratio1=RATIO1, Ratio2=RATIO2, NeuronsToTest=NEURONSTOTEST)

In [ ]:
RUN(IM, modelename, Dist='euclidean', Ratio1=RATIO1, Ratio2=RATIO2, NeuronsToTest=NEURONSTOTEST)